## Yggdrasil Model Visualization

### Libraries Used

In [ ]:
import jupyterlab_nodeeditor as jlne
import ipywidgets
import time
from yggdrasil.runner import YggRunner

### Optional Helper Function: Clean Bytes
Input: A 'bytes' type of data

Output: A cleaned 'string' conversion of the input

The reason behind this is just to make the output legible by removing quotations and any useless characters.
Note: Only works on some models

In [ ]:
def clean_bytes(ibytes):
    return str(ibytes)[1:-1].strip()

### Setup
Initialize the Node Editor instance 'ne' and load a model into it.

Use the filepath *sample_test_models/model_trifecta.yml* for a sample model.

As a note, the Trifecta sample model consists of three models
1. Model RNG - A model which outputs a random number 4 times (1 per line in the input text file)
2. Model Reader - Reads said input text file line by line
3. Model ROTXX - Rotates each line by the number generated and outputs it encoding in a ROT-XX format (think of ROT13 encoding, where each letter in the alphabet is moved 13 places, but here the number is random)

**Make sure you manually right click and add all of your desired components once the editor appears.**

While the rest of the notebook only needs 1 tab to run, it is important to notice how your model is loaded.

As of March 30th, 2022, there is a bug that does not allow for the components to be linked. We are working on fixing that but for production it will still execute normally.

In [ ]:
ne = jlne.NodeEditor()

# Change this to your model's filepath
model_filepath = "sample_test_models\model_trifecta.yml"

jlne.load_model(model_filepath, ne)

Right click the JLNE instance and open a new view in a seperate tab when you run the YggRunner below.

### Callback Classes and Display
This is where things get a bit messy **but** is still automated. The explanations may just seem difficult to understand, but I will do my best to simplify things and explain them in detail still.

The class name doesn't matter (*VariableDisplay* - Display the variable) so long as it is called properly. The comments in the code will explain each portion to better help the flow of this example.

*args[0]* is where the data in the runner is stored (i.e. the data that is being moved around, in the sample model's case, the number, line of text, and the rotated output)

### Connection Drivers
By having a runner on the model path, we can execute the YAML file directly inside the notebook. Please make sure that the variable "model_filepath" was changed correctly above.

In [ ]:
# Create a runner for the model, adjust the filepath for wherever the model is located
runner = YggRunner(model_filepath)

Below is the class that will update the JLNE instance in real time.

Read the comments for more details but the basic gist is that it will take each parsed component's arguments and output them as they are run in the first tab in the visualization

In [ ]:
# Create the callback class and update the first tab with live information

class VariableDisplay:
    # We add in filler labels to overwrite with actual data later since it is automated
    if len(ne.node_editor.nodes[0].display_element.children) <= 5:
        for i in range(len(list(runner.connectiondrivers))):
            ne.node_editor.nodes[0].display_element.children += (ipywidgets.Label("Initializing"),)
    
    # The initialization will have the label from ipywidgets but also the name of the variable and it's order in the list
    def __init__(self, elem_number, name):
        self.label = ipywidgets.Label()
        self.elem_number = elem_number
        self.name = name
    
    # Display data will always show the relevant information taken from the dictionary itself and use the default dictionary key as it's name
    # As of May 31st, 2022, the timesync bug still exists, so the time.sleep is necessary to see it run in realtime
    # Otherwise, it still works as intended
    # If you are only processing basic tings, you may opt to use the clean_bytes function with 'args[0].args' though it may be better to not use it as it may convolute the data being processed
    def display_data(self, *args, **kwargs):
        ne.node_editor.nodes[0].display_element.children[self.elem_number].value = f"{self.name}: {args[0].args}"
        time.sleep(1)

Each connectiondriver dictionary key corresponds to a connection in your model.
For the sample model, we have
1. The text file connected to the Reader model
2. The random number and line of text connected to the ROTXX model
3. The output of the ROTXX model (i.e. the rotated string)

To utilize realtime updating with a Yggdrasil Runner, we need to add in a 'callbacks' key to each connectiondriver. The value of this key will be the class function corresponding to the data you want to show casted to a list.

To break it down further, the value must be the class class with brackets around it so it is recognized by the Runner. This will allow the Runner to call back to that class function whenever the model is run. 

**This is all automated for you; there is no need to adjust any functions, just run the cells!**

In [ ]:
# We pull the exact drivers from the dictionary keys from the model itself here
for i, v in enumerate(list(runner.connectiondrivers)):
    display = VariableDisplay(i + 3, v.split(":")[1])
    runner.connectiondrivers[v]['callbacks'] = [display.display_data]

Finally we can execute the Runner to watch it all flow

In [ ]:
# Run the runner to see the data update in realtime
runner.run()

If you need to rerun, you must reload the YggRunner as well as everything after it (All of the Connection Drivers tab minus seeing the actual drivers)